# MIDTERM EXAM PREP

In [1]:
import numpy as np
import pandas as pd
import pulp
from pulp import *
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

### Stochastic Analysis

### Portfolio Allocation (dedication & immunization)

In [2]:
def gen_bond(tenor, coupon, price):
    '''
    Generates a dictionary representing the cashflows of a bond  
    ---
    Parameters:  
    tenor (int): periods of interest until maturity of the bond  
    coupon (float): coupon payment of bond  
    price (float): price of the bond  
    ---
    Returns:  
    bond (dict): dictionary showing bond cashflows by period
    '''
    cfs = []
    cfs.append(-price)
    for i in range(1,tenor):
        cfs.append(coupon)
    cfs.append(100+coupon)
    bond = dict(zip(range(tenor+1), cfs))
    return bond

In [3]:
def bond_names(bond_count):
    '''
    Generates list of bond names as strings formatted for LaTeX in Markdown  
    ---
    Parameters:  
    bond_count (int): number of bonds in list
    ---
    Returns:  
    names (list): list of bonds names as strings formatted for LaTeX math-mode
    '''
    names = []
    for i in range(1,bond_count+1):
        names.append('$b_{' + str(i) + '}$')
    return names

In [4]:
def bond_data(tenors, prices, coupons):
    '''
    Generates a DataFrame of the bond cashflows over time  
    ---
    Parameters:  
    tenors (list): List of bond tenors (ints)  
    prices (list): List of bond priuces (floats)  
    coupons (list): List of bond coupons (floats)
    ---
    Returns:  
    df (DataFrame): DataFrame of bond cashflows
        x - cash flow years  
        y - bond names (formatted for LaTeX)
    '''
    if len(tenors) == len(prices) == len(coupons):
        bond_count = len(tenors)
        bond_data = []
        for i in range(bond_count):
            bond_data.append(gen_bond(tenors[i], coupons[i], prices[i]))
        df = pd.DataFrame(bond_data, index=bond_names(bond_count)).fillna(0).transpose()
        df.index.name = 'cf_yr'
    else:
        print('ERROR: Tenors, Prices, and Coupons lists of different lengths')
    return df

In [5]:
def pv_factors(rates):
    '''
    Calculates Present Value Factors for given term structure of interest rates  
    ---
    Parameters:  
    rates (list): given term structure (float)  
    ---
    Returns:  
    pv (np array): PV Factors  
    '''
    pv = []
    for i in range(len(rates)):
        pv.append(1/(1+rates[i])**i)
    return np.asarray(pv)

In [6]:
def dur_factors(rates):
    '''
    Calculates Duration Factors for given term structure of interest rates  
    ---
    Parameters:  
    rates (list): given term structure (float)  
    ---
    Returns:  
    dur (np array): Duration Factors  
    '''
    dur = []
    for i in range(len(rates)):
        dur.append(i/(1+rates[i])**(i+1))
    return np.asarray(dur)

In [7]:
def conv_factors(rates):
    '''
    Calculates Convexity Factors for given term structure of interest rates  
    ---
    Parameters:  
    rates (list): given term structure (float)  
    ---
    Returns:  
    conv (np array): Convexity Factors  
    '''
    conv = []
    for i in range(len(rates)):
        conv.append((i*(i+1))/(1+rates[i])**(i+2))
    return np.asarray(conv)

In [8]:
def allocate_portfolio(liabilities, bonds_df, rates, r=0, immunization=False, duration=False, convexity=False, dedicated_years=0):
    '''
    Allocates Bond Portfolio by minimizing initial portfolio value subject to constraints (manipulatable)  
    ---
    Parameters:  
    liabilities (list): liability stream portfolio must cover (float)  
    bonds_df (DataFrame): bond data  
    r (float): interest rate on cash carry (default 0)  
    rates (list): current term structure of interest rates (float)  
    immunization (bool): solve the portfolio using present value immunized cashflows (default False)  
    duration (bool): solve the portfolio using duration immunized cashflows (default False)
    convexity (bool): adds convexity constraint to immunization solve (default False)  
    dedicated years (int): years dedicated to cashflow for immunized solve (default 0 in immunization)
    ---
    Returns:  
    Portfolio (LpProblem): Linear Program to be solved
    '''

    '''Problem Type and Set Up'''
    bonds = bonds_df.columns
    if immunization:
        years = dedicated_years
        print(years)
    else:
        years = max(bonds_df.index) + 1

    '''Decision Variables'''
    bond_alloc = LpVariable.dicts('Bonds',bonds,lowBound=0)
    carry = LpVariable.dicts('CashCarry',range(max(bonds_df.index)+1),lowBound=0)

    '''Objective Function'''
    portfolio = LpProblem('Allocation',LpMinimize)
    portfolio += lpSum([bonds_df[i][0]*bond_alloc[i]*(-1) for i in bonds] + carry[0])
    
    '''Constraint - Dedication Years - Dedication & Immunization'''
    if years > 0:
        for i in range(1,years):
            portfolio += lpSum([bonds_df[j][i] * bond_alloc[j] for j in bonds] + carry[i-1]*(1+r) - carry[i]) >= liabilities[i]
    else:
        pass
    
    '''Constraint - NPV - Immunization'''
    if immunization:
        portfolio += lpSum([bonds_df[i][1:] * pv_factors(rates)[1:] * bond_alloc[i] for i in bonds]) == sum(liabilities * pv_factors(rates))
    else:
        pass

    '''Constraint - Duration - Immunization'''
    if duration:
        portfolio += lpSum([bonds_df[i][1:] * dur_factors(rates)[1:] * bond_alloc[i] for i in bonds]) == sum(liabilities * dur_factors(rates))
    else:
        pass

    '''Constraint - Convexity - Immunization'''
    if convexity:
        portfolio += lpSum([bonds_df[i][1:] * conv_factors(rates)[1:] * bond_alloc[i] for i in bonds]) == sum(liabilities * conv_factors(rates))
    else:
        pass

    return portfolio

In [9]:
def solve_portfolio(portfolio, bonds_df):
    '''
    Solves a Given PuLP Optimization Program and Outputs the Optimal Solution, Variable Allocation, and Sensitivity Information  
    ---
    Parameters:  
    problem (LpProblem) - Formulated LP   
    df (DataFrame) - Problem DataFrame  
    ---
    Returns:  
    final_dec (DataFrame) - Decision Variable Final Values  
    sp_df (DataFrame) - Shadow Prices  
    '''

    '''Solves portfolio'''
    portfolio.solve()

    '''Decision Variable Values'''
    length = len(bonds_df.columns)
    var_alloc = dict(zip([v.name[6:] for v in portfolio.variables()[:length]], [v.varValue for v in portfolio.variables()[:length]]))    
    dec_vars = pd.DataFrame(var_alloc, index=['count']).transpose()
    
    '''Shadow Prices'''
    sp = [{'name':name, 'shadow_price':c.pi} for name, c in portfolio.constraints.items()]
    sp_df = pd.DataFrame(sp)

    '''Rediced Cost'''
    c = np.asarray(bonds_df.loc[0] * (-1))
    AT = bonds_df.transpose().iloc[:,1:].to_numpy()
    y = np.asarray(sp_df['shadow_price'])
    rc = np.round(c - np.matmul(AT,y),3)
    rc_df = pd.DataFrame(rc,index=bonds_df.columns)
    
    final_dec = pd.merge(dec_vars,rc_df, how='inner', left_index=True, right_index=True)
    final_dec.columns = ['var_val', 'reduced_cost']
    final_dec.index.name = 'Bond'
    
    return final_dec, sp_df
    

In [10]:
'''
PROBLEM DATA
'''

# Bonds
tenors = [6,6,6,5,5,4,4,3,3,3,2,2,1]
prices = [109, 94.8, 99.5, 93.1, 97.2, 96.3, 92.9, 110, 104, 101, 107, 102, 95.2]
coupons = [10,7,8,6,7,6,5,10,8,6,10,7,0]
if len(tenors) == len(prices) == len(coupons):
    length = len(tenors)
    cf_yr_max = max(tenors)
    bonds = bond_names(length)

bonds_df = bond_data(tenors,prices,coupons)

# Liabilities
liabilities = [0,500,200,800,200,800,1200]

# Rates
rates = [0,.0504,.0594,.0636,.0718,.0789,.0839]

In [11]:
portfolio = allocate_portfolio(liabilities,bonds_df,rates)

In [12]:
bonds_df

,$b_{1}$,$b_{2}$,$b_{3}$,$b_{4}$,$b_{5}$,$b_{6}$,$b_{7}$,$b_{8}$,$b_{9}$,$b_{10}$,$b_{11}$,$b_{12}$,$b_{13}$
cf_yr,,,,,,,,,,,,,
0,-109.0,-94.8,-99.5,-93.1,-97.2,-96.3,-92.9,-110.0,-104.0,-101.0,-107.0,-102.0,-95.2
1,10.0,7.0,8.0,6.0,7.0,6.0,5.0,10.0,8.0,6.0,10.0,7.0,100.0
2,10.0,7.0,8.0,6.0,7.0,6.0,5.0,10.0,8.0,6.0,110.0,107.0,0.0
3,10.0,7.0,8.0,6.0,7.0,6.0,5.0,110.0,108.0,106.0,0.0,0.0,0.0
4,10.0,7.0,8.0,6.0,7.0,106.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0
5,10.0,7.0,8.0,106.0,107.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,110.0,107.0,108.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
portfolio.variables()

[Bonds_$b_{10}$,
 Bonds_$b_{11}$,
 Bonds_$b_{12}$,
 Bonds_$b_{13}$,
 Bonds_$b_{1}$,
 Bonds_$b_{2}$,
 Bonds_$b_{3}$,
 Bonds_$b_{4}$,
 Bonds_$b_{5}$,
 Bonds_$b_{6}$,
 Bonds_$b_{7}$,
 Bonds_$b_{8}$,
 Bonds_$b_{9}$,
 CashCarry_0,
 CashCarry_1,
 CashCarry_2,
 CashCarry_3,
 CashCarry_4,
 CashCarry_5,
 CashCarry_6]

In [14]:
solve_portfolio(portfolio, bonds_df)

(            var_val  reduced_cost
 Bond                             
 $b_{10}$   0.000000         2.330
 $b_{11}$   0.108870        -0.000
 $b_{12}$   0.000000         0.515
 $b_{13}$   3.108870         0.000
 $b_{1}$    0.000000         0.052
 $b_{2}$    0.000000         0.024
 $b_{3}$   11.111111         0.000
 $b_{4}$    0.000000         0.007
 $b_{5}$    6.645898        -0.000
 $b_{6}$    0.609338         0.000
 $b_{7}$    0.000000         0.023
 $b_{8}$    0.000000         0.670
 $b_{9}$    6.119757        -0.000,
   name  shadow_price
 0  _C1      0.952000
 1  _C2      0.886182
 2  _C3      0.826801
 3  _C4      0.757642
 4  _C5      0.684501
 5  _C6      0.617065)